In [6]:
import os
import rasterio

In [ ]:
input_images = [img for img in os.listdir('images') if img.endswith('.jpg')]
print(input_images[:5])

inpath = 'images'
outpath = 'images/tif'

In [ ]:
if not os.path.isdir(outpath):
    os.makedirs(outpath)
    print('Directory was created')
else:
    print('Directory already exists')

In [ ]:
kwargs = {
    'driver': 'GTiff',
    'dtype': 'float32',
    'crs': 'EPSG:32721', 
    'nodata': 0,
    'compress': 'lzw',
    'count': 3
}

for img in input_images:
    input_file = os.path.join(inpath, img)
    output_file = os.path.join(outpath, img.replace('.jpg', '.tif'))

    raster = rasterio.open(input_file)

    out_meta = raster.meta.copy()
    out_meta.update(kwargs)

    with rasterio.open(
        output_file,
        mode= 'w',
        **out_meta
    ) as dst:
        dst.write_band(1, raster.read(1))
        dst.write_band(2, raster.read(2))
        dst.write_band(3, raster.read(3))

## Vector masking

In [7]:
import matplotlib.pyplot as plt
import numpy as np
from rasterio.plot import show
import geopandas as gpd
from rasterio import features

In [8]:
# Read in vector
vector = gpd.read_file("input_vector_data/labels.geojson")

# Get list of geometries for all features in vector file
geom = [shapes for shapes in vector.geometry]

In [ ]:
vector.plot()

In [ ]:
input_tif = 'images/tif/J-29-C-5-M-9.tif'

raster = rasterio.open(input_tif)

In [ ]:
# Function to normalize the grid values
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize the bands
redn = normalize(raster.read(1))
greenn = normalize(raster.read(2))
bluen = normalize(raster.read(3))

rgb = np.dstack((redn, greenn, bluen))

# Let's see how our color composite looks like
plt.imshow(rgb)

In [ ]:
metadata = raster.meta.copy()
metadata

In [ ]:
metadata['transform']

In [ ]:
# Rasterize vector using the shape and coordinate system of the raster
rasterized = features.rasterize(
    geom,
    out_shape = raster.shape,
    fill = 0,
    out = None,
    transform = raster.transform,
    all_touched = False,
    default_value = 1,
    dtype = None)

# Plot raster
fig, ax = plt.subplots(1, figsize = (10, 10))
show(rasterized, ax = ax)
plt.gca().invert_yaxis()

In [ ]:
metadata['crs']

In [ ]:
new_dataset = rasterio.open(
    'foo.tif',
    'w',
    driver='GTiff',
    height = rasterized.shape[0],
    width = rasterized.shape[1],
    count=1,
    dtype=str(rasterized.dtype),
    crs=metadata['crs'],
    transform=metadata['transform'])

new_dataset.write(rasterized, 1)
new_dataset.close()

In [9]:
vector = gpd.read_file("input_vector_data/labels.geojson")
geom = [shapes for shapes in vector.geometry]

def generate_mask(input_image, labels, output_image):
    raster = rasterio.open(input_image)
    metadata = raster.meta.copy()

    rasterized = features.rasterize(
        geom,
        out_shape = raster.shape,
        fill = 0,
        out = None,
        transform = raster.transform,
        all_touched = False,
        default_value = 1,
        dtype = None)
    
    if np.sum(rasterized)==0:
        print(f'No intersection between {input_image} and vector')
        return
    else:
        new_dataset = rasterio.open(
            output_image,
            'w',
            driver='GTiff',
            height=rasterized.shape[0],
            width=rasterized.shape[1],
            count=1,
            dtype=str(rasterized.dtype),
            crs=metadata['crs'],
            transform=metadata['transform'])

        new_dataset.write(rasterized, 1)
        new_dataset.close()
        print(f'Saved image to {output_image}')

In [10]:
input_images = [f'images/tif/{img}' for img in os.listdir('images/tif') if img.endswith('.tif')]
input_images[:5]

['images/tif/J-29-C-5-M-7.tif',
 'images/tif/J-29-C-4-P-4.tif',
 'images/tif/J-29-C-4-P-2.tif',
 'images/tif/J-29-C-4-P-3.tif',
 'images/tif/J-29-C-4-N-3.tif']

In [11]:
output_images = [f'{os.path.dirname(img)}/masks/{os.path.basename(img)}' for img in input_images]
output_images[:5]

['images/tif/masks/J-29-C-5-M-7.tif',
 'images/tif/masks/J-29-C-4-P-4.tif',
 'images/tif/masks/J-29-C-4-P-2.tif',
 'images/tif/masks/J-29-C-4-P-3.tif',
 'images/tif/masks/J-29-C-4-N-3.tif']

In [12]:
for input_img, output_img in zip(input_images, output_images):
    generate_mask(input_img, geom, output_img)

No intersection between images/tif/J-29-C-5-M-7.tif and vector
No intersection between images/tif/J-29-C-4-P-4.tif and vector
No intersection between images/tif/J-29-C-4-P-2.tif and vector
No intersection between images/tif/J-29-C-4-P-3.tif and vector
No intersection between images/tif/J-29-C-4-N-3.tif and vector
No intersection between images/tif/J-29-C-4-O-9.tif and vector
No intersection between images/tif/J-29-C-4-N-6.tif and vector
No intersection between images/tif/J-29-C-6-M-5.tif and vector
No intersection between images/tif/J-29-C-4-N-2.tif and vector
No intersection between images/tif/J-29-C-5-M-2.tif and vector
No intersection between images/tif/J-29-C-6-M-2.tif and vector
No intersection between images/tif/J-29-C-5-N-4.tif and vector
No intersection between images/tif/J-29-C-2-O-7.tif and vector
No intersection between images/tif/J-29-C-1-O-6.tif and vector
No intersection between images/tif/J-29-C-2-O-8.tif and vector
No intersection between images/tif/J-29-C-2-P-9.tif and